In [ ]:
import requests
import pandas as pd
import os
from tqdm import tqdm
from io import StringIO
import seaborn as sns
import numpy as np
from bs4 import BeautifulSoup
import re
import requests
import matplotlib.pyplot as plt
from datetime import datetime
from geopy.geocoders import Nominatim
import transformers
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [ ]:
bidendf = pd.read_csv('hashtag_joebiden.csv', engine='python')

In [ ]:
trumpdf = pd.read_csv('hashtag_donaldtrump.csv', engine='python')

In [ ]:
bidendf = bidendf[bidendf['state'].notnull() & bidendf['lat'].notnull() & bidendf['long'].notnull()]
trumpdf = trumpdf[trumpdf['state'].notnull() & trumpdf['lat'].notnull() & trumpdf['long'].notnull()]

In [ ]:
bidendf

,created_at,tweet_id,tweet,likes,retweet_count,source,user_id,user_name,user_screen_name,user_description,...,user_followers_count,user_location,lat,long,city,country,continent,state,state_code,collected_at
0,2020-10-15 00:00:01,1.316529221557252e+18,#Elecciones2020 | En #Florida: #JoeBiden dice ...,0.0,0.0,TweetDeck,360666534.0,El Sol Latino News,elsollatinonews,🌐 Noticias de interés para latinos de la costa...,...,1860.0,"Philadelphia, PA / Miami, FL",25.77427,-80.19366,NaN,United States of America,North America,Florida,FL,2020-10-21 00:00:00
2,2020-10-15 00:00:20,1.3165293050069524e+18,@IslandGirlPRV @BradBeauregardJ @MeidasTouch T...,0.0,0.0,Twitter Web App,3494182277.0,Flag Waver,Flag_Wavers,NaN,...,1536.0,Golden Valley Arizona,46.3040364,-109.17143119999999,NaN,United States of America,North America,Montana,MT,2020-10-21 00:00:01.035654566
4,2020-10-15 00:00:22,1.316529312741253e+18,#censorship #HunterBiden #Biden #BidenEmails #...,1.0,0.0,Twitter Web App,1.032806955356545e+18,the Gold State,theegoldstate,A Silicon Valley #independent #News #Media #St...,...,390.0,"California, USA",36.7014631,-118.75599740000001,NaN,United States of America,North America,California,CA,2020-10-21 00:00:02.071309132
6,2020-10-15 00:00:25,1.3165293244182405e+18,"In 2020, #NYPost is being #censorship #CENSORE...",0.0,0.0,Twitter for iPhone,19940334.0,Change Illinois | Biden will increase taxes by...,changeillinois,"Illinois, home of Lincoln and Reagan, used to ...",...,1397.0,"Chicago, Illinois",41.875561600000005,-87.6244212,Chicago,United States of America,North America,Illinois,IL,2020-10-21 00:00:03.106963698
11,2020-10-15 00:00:57,1.3165294600327987e+18,"FBI Allegedly Obtained Hunter Biden Computer, ...",0.0,0.0,Twitter for Android,9.607387225967534e+17,RLCompton,NewfoundStudio,"""There are things known and there are things u...",...,266.0,"Kentucky, USA",37.5726028,-85.1551411,NaN,United States of America,North America,Kentucky,KY,2020-10-21 00:00:05.696100113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
777057,2020-11-08 23:58:48,1.3255886120843305e+18,The man needs some help...#usa #biden\nWhen wi...,0.0,0.0,Twitter for Android,1.2480465028214456e+18,Dr J,DrJoeMcCarthy,Human. Free Thinker. Met Mandela. Personal. Fa...,...,311.0,Earth. 3rd Planet from Sun.,43.51963,-114.31532,NaN,United States of America,North America,Idaho,ID,2020-11-09 18:32:45.641087
777058,2020-11-08 23:58:52,1.3255886292516127e+18,Şu #biden geleli 1 gün oldu #maliyebakanı #ist...,0.0,0.0,Twitter for Android,1.317045546616578e+18,MrErdem,MrErdem23,hayat yaşamaya değer,...,0.0,"İzmir, Türkiye",38.4147331,27.1434119,Izmir,Turkey,Asia,Izmir,NaN,2020-11-09 18:32:45.567901
777059,2020-11-08 23:58:56,1.3255886468718387e+18,LIONZ DEN PRESENTS TO YOU \n\n“THE WHITE OBAMA...,1.0,0.0,Twitter for Android,9.868224530821652e+17,IUIC Minnesota,IUICMinnesota,NaN,...,674.0,"Minnesota, USA",45.98965870000001,-94.6113288,NaN,United States of America,North America,Minnesota,MN,2020-11-09 18:32:45.794545
777063,2020-11-08 23:59:16,1.325588732318265e+18,"Mr. #Biden, tear down that wall (with #Mexico)...",1.0,0.0,Twitter for iPhone,377229570.0,Tim Welch,TimFWelch,transportation nerd; data geek.\nsenior lectur...,...,444.0,"Auckland, NZ",-36.852095,174.76318030000002,Auckland,New Zealand,Oceania,Auckland,AUK,2020-11-09 18:32:45.760347


In [ ]:
trumpdf

,created_at,tweet_id,tweet,likes,retweet_count,source,user_id,user_name,user_screen_name,user_description,...,user_followers_count,user_location,lat,long,city,country,continent,state,state_code,collected_at
0,2020-10-15 00:00:01,1.316529221557252e+18,#Elecciones2020 | En #Florida: #JoeBiden dice ...,0.0,0.0,TweetDeck,360666534.0,El Sol Latino News,elsollatinonews,🌐 Noticias de interés para latinos de la costa...,...,1860.0,"Philadelphia, PA / Miami, FL",25.77427,-80.19366,NaN,United States of America,North America,Florida,FL,2020-10-21 00:00:00
2,2020-10-15 00:00:02,1.316529228091847e+18,"#Trump: As a student I used to hear for years,...",2.0,1.0,Twitter Web App,8436472.0,snarke,snarke,"Will mock for food! Freelance writer, blogger,...",...,1185.0,Portland,45.5202471,-122.6741949,Portland,United States of America,North America,Oregon,OR,2020-10-21 00:00:00.746433060
4,2020-10-15 00:00:08,1.3165292523014513e+18,You get a tie! And you get a tie! #Trump ‘s ra...,4.0,3.0,Twitter for iPhone,47413798.0,Rana Abtar - رنا أبتر,Ranaabtar,"Washington Correspondent, Lebanese-American ,c...",...,5393.0,Washington DC,38.8949924,-77.0365581,Washington,United States of America,North America,District of Columbia,DC,2020-10-21 00:00:01.492866121
5,2020-10-15 00:00:17,1.316529291052675e+18,@CLady62 Her 15 minutes were over long time ag...,2.0,0.0,Twitter for Android,1138416104.0,Farris Flagg,FarrisFlagg,#BidenHarris2020 #JoeBiden2020 #KamalaHarrisFo...,...,2363.0,"Perris,California",33.7825194,-117.22864779999999,NaN,United States of America,North America,California,CA,2020-10-21 00:00:01.866082651
7,2020-10-15 00:00:18,1.3165292934979625e+18,@DeeviousDenise @realDonaldTrump @nypost There...,0.0,0.0,Twitter for iPhone,9.007610716314296e+17,Stacey Gulledge 🇺🇸 Patriot ♥️ KAG 🙏 👮‍♀️♥️,sm_gulledge,"Patriot, Wife, “Shaken not Stirred” Mom of two...",...,766.0,"Ohio, USA",40.225356899999994,-82.6881395,NaN,United States of America,North America,Ohio,OH,2020-10-21 00:00:02.612515712
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
971073,2020-11-08 23:59:21,1.3255887529331958e+18,"Again with the lying, cheating, b.s. #LoserTru...",0.0,0.0,Twitter Web App,28423445.0,The Honorable Terri Lee Watson,dancertav,"I'm green, lean and a dancing machine. I draw,...",...,1197.0,South Florida,37.1677128,-94.84642559999999,NaN,United States of America,North America,Kansas,KS,2020-11-09 17:47:56.062661
971075,2020-11-08 23:59:25,1.325588768448e+18,"Donald Trump, Moscow, 1996, shared by @oooroma...",8.0,3.0,Twitter Web App,388137441.0,Elisabeth Schimpfössl,ElisabethSchimp,Author of Rich Russians,...,586.0,"London, England",51.5073219,-0.1276474,London,United Kingdom,Europe,England,ENG,2020-11-09 17:47:55.816241
971078,2020-11-08 23:59:36,1.3255888134066708e+18,13 emails from #Trump and Trumpettes today. 20...,1.0,0.0,Twitter for iPhone,357849201.0,murdoch davis,murdochdavis,"I try to be pithy, not pissy. Like you I fail....",...,1190.0,Valparaiso IN Team @mlh_holmes,-33.0458456,-71.6196749,Valparaíso,Chile,South America,Valparaiso Region,NaN,2020-11-09 17:47:56.022322
971079,2020-11-08 23:59:37,1.325588820495016e+18,"unlike MUSSOLINI, HITLER, MAO, LENIN &amp; STA...",0.0,0.0,Twitter Web App,313536147.0,ActivistBowen,ActivistBowen2,"person, web writer, dissident, son, brother, f...",...,3792.0,Hong Kong,22.2793278,114.16281310000001,Hong Kong Island,China,Asia,Hong Kong,NaN,2020-11-09 17:47:56.080455


In [ ]:
bidendf.nunique()

created_at              215170
tweet_id                259671
tweet                   257849
likes                     1152
retweet_count              538
source                     343
user_id                  98156
user_name                94895
user_screen_name         98266
user_description         92562
user_join_date           98133
user_followers_count     20903
user_location            11500
lat                       5037
long                      5040
city                      1267
country                    121
continent                    8
state                      639
state_code                 294
collected_at            260188
dtype: int64

In [ ]:
trumpdf.nunique()

created_at              280871
tweet_id                320309
tweet                   316976
likes                     1072
retweet_count              486
source                     397
user_id                  87872
user_name                85312
user_screen_name         87997
user_description         82962
user_join_date           87861
user_followers_count     21541
user_location            11688
lat                       5062
long                      5064
city                      1310
country                    126
continent                    8
state                      668
state_code                 297
collected_at            320607
dtype: int64

In [ ]:
final_df = pd.concat([trumpdf, bidendf], ignore_index=True)

In [ ]:
final_df

,created_at,tweet_id,tweet,likes,retweet_count,source,user_id,user_name,user_screen_name,user_description,...,user_followers_count,user_location,lat,long,city,country,continent,state,state_code,collected_at
0,2020-10-15 00:00:01,1.316529221557252e+18,#Elecciones2020 | En #Florida: #JoeBiden dice ...,0.0,0.0,TweetDeck,360666534.0,El Sol Latino News,elsollatinonews,🌐 Noticias de interés para latinos de la costa...,...,1860.0,"Philadelphia, PA / Miami, FL",25.77427,-80.19366,NaN,United States of America,North America,Florida,FL,2020-10-21 00:00:00
1,2020-10-15 00:00:02,1.316529228091847e+18,"#Trump: As a student I used to hear for years,...",2.0,1.0,Twitter Web App,8436472.0,snarke,snarke,"Will mock for food! Freelance writer, blogger,...",...,1185.0,Portland,45.5202471,-122.6741949,Portland,United States of America,North America,Oregon,OR,2020-10-21 00:00:00.746433060
2,2020-10-15 00:00:08,1.3165292523014513e+18,You get a tie! And you get a tie! #Trump ‘s ra...,4.0,3.0,Twitter for iPhone,47413798.0,Rana Abtar - رنا أبتر,Ranaabtar,"Washington Correspondent, Lebanese-American ,c...",...,5393.0,Washington DC,38.8949924,-77.0365581,Washington,United States of America,North America,District of Columbia,DC,2020-10-21 00:00:01.492866121
3,2020-10-15 00:00:17,1.316529291052675e+18,@CLady62 Her 15 minutes were over long time ag...,2.0,0.0,Twitter for Android,1138416104.0,Farris Flagg,FarrisFlagg,#BidenHarris2020 #JoeBiden2020 #KamalaHarrisFo...,...,2363.0,"Perris,California",33.7825194,-117.22864779999999,NaN,United States of America,North America,California,CA,2020-10-21 00:00:01.866082651
4,2020-10-15 00:00:18,1.3165292934979625e+18,@DeeviousDenise @realDonaldTrump @nypost There...,0.0,0.0,Twitter for iPhone,9.007610716314296e+17,Stacey Gulledge 🇺🇸 Patriot ♥️ KAG 🙏 👮‍♀️♥️,sm_gulledge,"Patriot, Wife, “Shaken not Stirred” Mom of two...",...,766.0,"Ohio, USA",40.225356899999994,-82.6881395,NaN,United States of America,North America,Ohio,OH,2020-10-21 00:00:02.612515712
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
580794,2020-11-08 23:58:48,1.3255886120843305e+18,The man needs some help...#usa #biden\nWhen wi...,0.0,0.0,Twitter for Android,1.2480465028214456e+18,Dr J,DrJoeMcCarthy,Human. Free Thinker. Met Mandela. Personal. Fa...,...,311.0,Earth. 3rd Planet from Sun.,43.51963,-114.31532,NaN,United States of America,North America,Idaho,ID,2020-11-09 18:32:45.641087
580795,2020-11-08 23:58:52,1.3255886292516127e+18,Şu #biden geleli 1 gün oldu #maliyebakanı #ist...,0.0,0.0,Twitter for Android,1.317045546616578e+18,MrErdem,MrErdem23,hayat yaşamaya değer,...,0.0,"İzmir, Türkiye",38.4147331,27.1434119,Izmir,Turkey,Asia,Izmir,NaN,2020-11-09 18:32:45.567901
580796,2020-11-08 23:58:56,1.3255886468718387e+18,LIONZ DEN PRESENTS TO YOU \n\n“THE WHITE OBAMA...,1.0,0.0,Twitter for Android,9.868224530821652e+17,IUIC Minnesota,IUICMinnesota,NaN,...,674.0,"Minnesota, USA",45.98965870000001,-94.6113288,NaN,United States of America,North America,Minnesota,MN,2020-11-09 18:32:45.794545
580797,2020-11-08 23:59:16,1.325588732318265e+18,"Mr. #Biden, tear down that wall (with #Mexico)...",1.0,0.0,Twitter for iPhone,377229570.0,Tim Welch,TimFWelch,transportation nerd; data geek.\nsenior lectur...,...,444.0,"Auckland, NZ",-36.852095,174.76318030000002,Auckland,New Zealand,Oceania,Auckland,AUK,2020-11-09 18:32:45.760347


In [ ]:
final_df.nunique()

created_at              405515
tweet_id                509930
tweet                   505408
likes                     1510
retweet_count              675
source                     452
user_id                 145507
user_name               139760
user_screen_name        145708
user_description        136893
user_join_date          145465
user_followers_count     30385
user_location            14455
lat                       6061
long                      6063
city                      1401
country                    128
continent                    8
state                      715
state_code                 308
collected_at            580794
dtype: int64

In [ ]:
final_df = final_df.drop_duplicates(subset='tweet_id')

In [ ]:
final_df

,created_at,tweet_id,tweet,likes,retweet_count,source,user_id,user_name,user_screen_name,user_description,...,user_followers_count,user_location,lat,long,city,country,continent,state,state_code,collected_at
0,2020-10-15 00:00:01,1.316529221557252e+18,#Elecciones2020 | En #Florida: #JoeBiden dice ...,0.0,0.0,TweetDeck,360666534.0,El Sol Latino News,elsollatinonews,🌐 Noticias de interés para latinos de la costa...,...,1860.0,"Philadelphia, PA / Miami, FL",25.77427,-80.19366,NaN,United States of America,North America,Florida,FL,2020-10-21 00:00:00
1,2020-10-15 00:00:02,1.316529228091847e+18,"#Trump: As a student I used to hear for years,...",2.0,1.0,Twitter Web App,8436472.0,snarke,snarke,"Will mock for food! Freelance writer, blogger,...",...,1185.0,Portland,45.5202471,-122.6741949,Portland,United States of America,North America,Oregon,OR,2020-10-21 00:00:00.746433060
2,2020-10-15 00:00:08,1.3165292523014513e+18,You get a tie! And you get a tie! #Trump ‘s ra...,4.0,3.0,Twitter for iPhone,47413798.0,Rana Abtar - رنا أبتر,Ranaabtar,"Washington Correspondent, Lebanese-American ,c...",...,5393.0,Washington DC,38.8949924,-77.0365581,Washington,United States of America,North America,District of Columbia,DC,2020-10-21 00:00:01.492866121
3,2020-10-15 00:00:17,1.316529291052675e+18,@CLady62 Her 15 minutes were over long time ag...,2.0,0.0,Twitter for Android,1138416104.0,Farris Flagg,FarrisFlagg,#BidenHarris2020 #JoeBiden2020 #KamalaHarrisFo...,...,2363.0,"Perris,California",33.7825194,-117.22864779999999,NaN,United States of America,North America,California,CA,2020-10-21 00:00:01.866082651
4,2020-10-15 00:00:18,1.3165292934979625e+18,@DeeviousDenise @realDonaldTrump @nypost There...,0.0,0.0,Twitter for iPhone,9.007610716314296e+17,Stacey Gulledge 🇺🇸 Patriot ♥️ KAG 🙏 👮‍♀️♥️,sm_gulledge,"Patriot, Wife, “Shaken not Stirred” Mom of two...",...,766.0,"Ohio, USA",40.225356899999994,-82.6881395,NaN,United States of America,North America,Ohio,OH,2020-10-21 00:00:02.612515712
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
580792,2020-11-08 23:58:27,1.3255885261178757e+18,#Nationals invite #Biden https://t.co/UVEs3CB6mM,0.0,0.0,Twitter Web App,4571814261.0,Devil In Disguise,DisguiseDevilin,Je souffre d'une allergie à la Bêtise Humaine!...,...,3230.0,Terre/Earth,8.5322391,38.626471200000005,NaN,Ethiopia,Africa,Oromia Region,NaN,2020-11-09 18:32:45.557984
580793,2020-11-08 23:58:46,1.3255886042492723e+18,No puede ser lo #Panazis como siempre sacando ...,0.0,0.0,Twitter for iPhone,7.40635438478594e+17,JeanCarlo,CarlosValenz15,Siempre en contra de la corrupción con la firm...,...,2313.0,"Tijuana, Baja California",32.501018800000004,-116.9646629,Tijuana,Mexico,North America,Baja California,BCN,2020-11-09 18:32:45.862885
580794,2020-11-08 23:58:48,1.3255886120843305e+18,The man needs some help...#usa #biden\nWhen wi...,0.0,0.0,Twitter for Android,1.2480465028214456e+18,Dr J,DrJoeMcCarthy,Human. Free Thinker. Met Mandela. Personal. Fa...,...,311.0,Earth. 3rd Planet from Sun.,43.51963,-114.31532,NaN,United States of America,North America,Idaho,ID,2020-11-09 18:32:45.641087
580795,2020-11-08 23:58:52,1.3255886292516127e+18,Şu #biden geleli 1 gün oldu #maliyebakanı #ist...,0.0,0.0,Twitter for Android,1.317045546616578e+18,MrErdem,MrErdem23,hayat yaşamaya değer,...,0.0,"İzmir, Türkiye",38.4147331,27.1434119,Izmir,Turkey,Asia,Izmir,NaN,2020-11-09 18:32:45.567901


In [ ]:
final_df.nunique()

created_at              405514
tweet_id                509930
tweet                   505403
likes                     1445
retweet_count              657
source                     451
user_id                 145504
user_name               139746
user_screen_name        145704
user_description        136862
user_join_date          145462
user_followers_count     28302
user_location            14454
lat                       6060
long                      6062
city                      1400
country                    127
continent                    7
state                      712
state_code                 308
collected_at            509929
dtype: int64

In [ ]:
usersdf = pd.read_csv('users.csv')

<ipython-input-721-2026fc4ba06c>:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  usersdf = pd.read_csv('users.csv')


In [ ]:
usersdf

,user_id,user_followers_count,state,city,country,is_politician,political_affiliation
0,RepDarrenSoto,NaN,Florida,NaN,United States of America,Yes,Democrat
1,RepJackyRosen,NaN,Nevada,NaN,United States of America,Yes,Democrat
2,RepAlLawsonJr,NaN,Florida,NaN,United States of America,Yes,Democrat
3,RepEspaillat,NaN,New York,NaN,United States of America,Yes,Democrat
4,RepBRochester,NaN,Delaware,NaN,United States of America,Yes,Democrat
...,...,...,...,...,...,...,...
421523,zzzbeeme,111.0,NaN,NaN,NaN,No,NaN
421524,zzzein26,2107.0,NaN,NaN,NaN,No,NaN
421525,zzzubair,416.0,NaN,NaN,India,No,NaN
421526,zzzz_accordd,356.0,NaN,NaN,NaN,No,NaN


In [ ]:
usersdf.rename(columns={'user_id':'user_handle'}, inplace=True)

In [ ]:
politiciandf = usersdf[usersdf['is_politician'] == 'Yes']

In [ ]:
politiciandf = politiciandf.sort_values(by='user_handle')

In [ ]:
politiciandf['user_id'] = range(len(politiciandf))

In [ ]:
politiciandf

,user_handle,user_followers_count,state,city,country,is_politician,political_affiliation,user_id
146,AGBecerra,NaN,California,NaN,United States of America,Yes,Democrat,0
173,AlanGrayson,NaN,Florida,NaN,United States of America,Yes,Democrat,1
39,AnthonyBrownMD4,NaN,Maryland,NaN,United States of America,Yes,Democrat,2
345,AustinScottGA08,NaN,Georgia,NaN,United States of America,Yes,Republican,3
162,BennieGThompson,NaN,Mississippi,NaN,United States of America,Yes,Democrat,4
...,...,...,...,...,...,...,...,...
296,reppittenger,NaN,North Carolina,NaN,United States of America,Yes,Republican,428
189,repsandylevin,NaN,Michigan,NaN,United States of America,Yes,Democrat,429
143,rosadelauro,NaN,Connecticut,NaN,United States of America,Yes,Democrat,430
110,sethmoulton,NaN,Massachusetts,NaN,United States of America,Yes,Democrat,431


In [ ]:
nonpoliticiandf =  usersdf[usersdf['is_politician'] != 'Yes']

In [ ]:
nonpoliticiandf.sort_values(by='user_handle')

,user_handle,user_followers_count,state,city,country,is_politician,political_affiliation
433,0000000ef,0.0,NaN,NaN,NaN,No,NaN
434,0000StingRay,451.0,NaN,NaN,NaN,No,NaN
435,00010001b,41.0,NaN,NaN,NaN,No,NaN
436,00010101010010O,0.0,NaN,NaN,NaN,No,NaN
437,00010111_,782.0,NaN,NaN,NaN,No,NaN
...,...,...,...,...,...,...,...
421523,zzzbeeme,111.0,NaN,NaN,NaN,No,NaN
421524,zzzein26,2107.0,NaN,NaN,NaN,No,NaN
421525,zzzubair,416.0,NaN,NaN,India,No,NaN
421526,zzzz_accordd,356.0,NaN,NaN,NaN,No,NaN


In [ ]:
nonpoliticiandf.nunique()

user_handle              421095
user_followers_count      23031
state                       710
city                       1507
country                     189
is_politician                 1
political_affiliation         0
dtype: int64

In [ ]:
final_df

,created_at,tweet_id,tweet,likes,retweet_count,source,user_id,user_name,user_screen_name,user_description,...,user_followers_count,user_location,lat,long,city,country,continent,state,state_code,collected_at
0,2020-10-15 00:00:01,1.316529221557252e+18,#Elecciones2020 | En #Florida: #JoeBiden dice ...,0.0,0.0,TweetDeck,360666534.0,El Sol Latino News,elsollatinonews,🌐 Noticias de interés para latinos de la costa...,...,1860.0,"Philadelphia, PA / Miami, FL",25.77427,-80.19366,NaN,United States of America,North America,Florida,FL,2020-10-21 00:00:00
1,2020-10-15 00:00:02,1.316529228091847e+18,"#Trump: As a student I used to hear for years,...",2.0,1.0,Twitter Web App,8436472.0,snarke,snarke,"Will mock for food! Freelance writer, blogger,...",...,1185.0,Portland,45.5202471,-122.6741949,Portland,United States of America,North America,Oregon,OR,2020-10-21 00:00:00.746433060
2,2020-10-15 00:00:08,1.3165292523014513e+18,You get a tie! And you get a tie! #Trump ‘s ra...,4.0,3.0,Twitter for iPhone,47413798.0,Rana Abtar - رنا أبتر,Ranaabtar,"Washington Correspondent, Lebanese-American ,c...",...,5393.0,Washington DC,38.8949924,-77.0365581,Washington,United States of America,North America,District of Columbia,DC,2020-10-21 00:00:01.492866121
3,2020-10-15 00:00:17,1.316529291052675e+18,@CLady62 Her 15 minutes were over long time ag...,2.0,0.0,Twitter for Android,1138416104.0,Farris Flagg,FarrisFlagg,#BidenHarris2020 #JoeBiden2020 #KamalaHarrisFo...,...,2363.0,"Perris,California",33.7825194,-117.22864779999999,NaN,United States of America,North America,California,CA,2020-10-21 00:00:01.866082651
4,2020-10-15 00:00:18,1.3165292934979625e+18,@DeeviousDenise @realDonaldTrump @nypost There...,0.0,0.0,Twitter for iPhone,9.007610716314296e+17,Stacey Gulledge 🇺🇸 Patriot ♥️ KAG 🙏 👮‍♀️♥️,sm_gulledge,"Patriot, Wife, “Shaken not Stirred” Mom of two...",...,766.0,"Ohio, USA",40.225356899999994,-82.6881395,NaN,United States of America,North America,Ohio,OH,2020-10-21 00:00:02.612515712
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
580792,2020-11-08 23:58:27,1.3255885261178757e+18,#Nationals invite #Biden https://t.co/UVEs3CB6mM,0.0,0.0,Twitter Web App,4571814261.0,Devil In Disguise,DisguiseDevilin,Je souffre d'une allergie à la Bêtise Humaine!...,...,3230.0,Terre/Earth,8.5322391,38.626471200000005,NaN,Ethiopia,Africa,Oromia Region,NaN,2020-11-09 18:32:45.557984
580793,2020-11-08 23:58:46,1.3255886042492723e+18,No puede ser lo #Panazis como siempre sacando ...,0.0,0.0,Twitter for iPhone,7.40635438478594e+17,JeanCarlo,CarlosValenz15,Siempre en contra de la corrupción con la firm...,...,2313.0,"Tijuana, Baja California",32.501018800000004,-116.9646629,Tijuana,Mexico,North America,Baja California,BCN,2020-11-09 18:32:45.862885
580794,2020-11-08 23:58:48,1.3255886120843305e+18,The man needs some help...#usa #biden\nWhen wi...,0.0,0.0,Twitter for Android,1.2480465028214456e+18,Dr J,DrJoeMcCarthy,Human. Free Thinker. Met Mandela. Personal. Fa...,...,311.0,Earth. 3rd Planet from Sun.,43.51963,-114.31532,NaN,United States of America,North America,Idaho,ID,2020-11-09 18:32:45.641087
580795,2020-11-08 23:58:52,1.3255886292516127e+18,Şu #biden geleli 1 gün oldu #maliyebakanı #ist...,0.0,0.0,Twitter for Android,1.317045546616578e+18,MrErdem,MrErdem23,hayat yaşamaya değer,...,0.0,"İzmir, Türkiye",38.4147331,27.1434119,Izmir,Turkey,Asia,Izmir,NaN,2020-11-09 18:32:45.567901


In [ ]:
final_df = final_df[['tweet_id', 'user_id', 'user_screen_name', 'created_at', 'tweet', 'likes', 'retweet_count', 'lat', 'long',  'state', 'state_code', 'country']].copy()
final_df.rename(columns={'user_screen_name' : 'user_handle'}, inplace=True)

In [ ]:
final_df

,tweet_id,user_id,user_handle,created_at,tweet,likes,retweet_count,lat,long,state,state_code,country
0,1.316529221557252e+18,360666534.0,elsollatinonews,2020-10-15 00:00:01,#Elecciones2020 | En #Florida: #JoeBiden dice ...,0.0,0.0,25.77427,-80.19366,Florida,FL,United States of America
1,1.316529228091847e+18,8436472.0,snarke,2020-10-15 00:00:02,"#Trump: As a student I used to hear for years,...",2.0,1.0,45.5202471,-122.6741949,Oregon,OR,United States of America
2,1.3165292523014513e+18,47413798.0,Ranaabtar,2020-10-15 00:00:08,You get a tie! And you get a tie! #Trump ‘s ra...,4.0,3.0,38.8949924,-77.0365581,District of Columbia,DC,United States of America
3,1.316529291052675e+18,1138416104.0,FarrisFlagg,2020-10-15 00:00:17,@CLady62 Her 15 minutes were over long time ag...,2.0,0.0,33.7825194,-117.22864779999999,California,CA,United States of America
4,1.3165292934979625e+18,9.007610716314296e+17,sm_gulledge,2020-10-15 00:00:18,@DeeviousDenise @realDonaldTrump @nypost There...,0.0,0.0,40.225356899999994,-82.6881395,Ohio,OH,United States of America
...,...,...,...,...,...,...,...,...,...,...,...,...
580792,1.3255885261178757e+18,4571814261.0,DisguiseDevilin,2020-11-08 23:58:27,#Nationals invite #Biden https://t.co/UVEs3CB6mM,0.0,0.0,8.5322391,38.626471200000005,Oromia Region,NaN,Ethiopia
580793,1.3255886042492723e+18,7.40635438478594e+17,CarlosValenz15,2020-11-08 23:58:46,No puede ser lo #Panazis como siempre sacando ...,0.0,0.0,32.501018800000004,-116.9646629,Baja California,BCN,Mexico
580794,1.3255886120843305e+18,1.2480465028214456e+18,DrJoeMcCarthy,2020-11-08 23:58:48,The man needs some help...#usa #biden\nWhen wi...,0.0,0.0,43.51963,-114.31532,Idaho,ID,United States of America
580795,1.3255886292516127e+18,1.317045546616578e+18,MrErdem23,2020-11-08 23:58:52,Şu #biden geleli 1 gün oldu #maliyebakanı #ist...,0.0,0.0,38.4147331,27.1434119,Izmir,NaN,Turkey


In [ ]:
final_df.nunique()

tweet_id         509930
user_id          145504
user_handle      145704
created_at       405514
tweet            505403
likes              1445
retweet_count       657
lat                6060
long               6062
state               712
state_code          308
country             127
dtype: int64

In [ ]:
nonpoliticiandf = pd.merge(nonpoliticiandf, final_df[['user_handle', 'user_id']], on='user_handle', how='left')

In [ ]:
nonpoliticiandf.nunique()

user_handle              421095
user_followers_count      23031
state                       710
city                       1507
country                     189
is_politician                 1
political_affiliation         0
user_id                  129389
dtype: int64

In [ ]:
final_df.isnull().sum()

tweet_id             0
user_id              0
user_handle          0
created_at           0
tweet                0
likes                0
retweet_count        0
lat                  0
long                 0
state                0
state_code       29611
country              0
dtype: int64

In [ ]:
nonpoliticiandf.isnull().sum()

user_handle                   0
user_followers_count          0
state                    291623
city                     453352
country                  247257
is_politician                 0
political_affiliation    781703
user_id                  291539
dtype: int64

In [ ]:
nonpoliticiandf

,user_handle,user_followers_count,state,city,country,is_politician,political_affiliation,user_id
0,0000000ef,0.0,NaN,NaN,NaN,No,NaN,NaN
1,0000StingRay,451.0,NaN,NaN,NaN,No,NaN,NaN
2,00010001b,41.0,NaN,NaN,NaN,No,NaN,NaN
3,00010101010010O,0.0,NaN,NaN,NaN,No,NaN,NaN
4,00010111_,782.0,NaN,NaN,NaN,No,NaN,NaN
...,...,...,...,...,...,...,...,...
781698,zzzbeeme,111.0,NaN,NaN,NaN,No,NaN,NaN
781699,zzzein26,2107.0,NaN,NaN,NaN,No,NaN,NaN
781700,zzzubair,416.0,NaN,NaN,India,No,NaN,NaN
781701,zzzz_accordd,356.0,NaN,NaN,NaN,No,NaN,NaN


In [ ]:
nonpoliticiandf = nonpoliticiandf.drop_duplicates()

In [ ]:
nonpoliticiandf

,user_handle,user_followers_count,state,city,country,is_politician,political_affiliation,user_id
0,0000000ef,0.0,NaN,NaN,NaN,No,NaN,NaN
1,0000StingRay,451.0,NaN,NaN,NaN,No,NaN,NaN
2,00010001b,41.0,NaN,NaN,NaN,No,NaN,NaN
3,00010101010010O,0.0,NaN,NaN,NaN,No,NaN,NaN
4,00010111_,782.0,NaN,NaN,NaN,No,NaN,NaN
...,...,...,...,...,...,...,...,...
781698,zzzbeeme,111.0,NaN,NaN,NaN,No,NaN,NaN
781699,zzzein26,2107.0,NaN,NaN,NaN,No,NaN,NaN
781700,zzzubair,416.0,NaN,NaN,India,No,NaN,NaN
781701,zzzz_accordd,356.0,NaN,NaN,NaN,No,NaN,NaN


In [ ]:
nonpoliticiandf = nonpoliticiandf.dropna(subset=['user_id'])
nonpoliticiandf = nonpoliticiandf.dropna(subset=['state'])

In [ ]:
nonpoliticiandf

,user_handle,user_followers_count,state,city,country,is_politician,political_affiliation,user_id
10,000HMY,42.0,Nevada,Las Vegas,United States of America,No,NaN,22564173.0
31,000alotaibi,44925.0,Riyadh Region,Ar Riyad,Saudi Arabia,No,NaN,453286951.0
36,001964kelly,1343.0,Ontario,NaN,Canada,No,NaN,342312372.0
40,001Newway,22.0,Texas,NaN,United States of America,No,NaN,9.56629717851009e+17
43,001annkavanagh,855.0,Leinster,Dublin,Ireland,No,NaN,24685294.0
...,...,...,...,...,...,...,...,...
781684,zzhuffxx,75.0,California,NaN,United States of America,No,NaN,2545204974.0
781688,zzo_pi,80.0,New York,NaN,United States of America,No,NaN,1.3075129524053238e+18
781689,zzpop_5,7.0,Texas,Austin,United States of America,No,NaN,3158296596.0
781694,zzxyxy,20997.0,California,NaN,United States of America,No,NaN,48883553.0


In [ ]:
nonpoliticiandf = nonpoliticiandf.drop_duplicates(subset='user_id')

In [ ]:
nonpoliticiandf.isnull().sum()

user_handle                   0
user_followers_count          0
state                         0
city                      39530
country                       0
is_politician                 0
political_affiliation    129339
user_id                       0
dtype: int64

In [ ]:
nonpoliticiandf.nunique()

user_handle              129338
user_followers_count      13738
state                       710
city                       1396
country                     129
is_politician                 1
political_affiliation         0
user_id                  129339
dtype: int64

In [ ]:
politiciandf

,user_handle,user_followers_count,state,city,country,is_politician,political_affiliation,user_id
146,AGBecerra,NaN,California,NaN,United States of America,Yes,Democrat,0
173,AlanGrayson,NaN,Florida,NaN,United States of America,Yes,Democrat,1
39,AnthonyBrownMD4,NaN,Maryland,NaN,United States of America,Yes,Democrat,2
345,AustinScottGA08,NaN,Georgia,NaN,United States of America,Yes,Republican,3
162,BennieGThompson,NaN,Mississippi,NaN,United States of America,Yes,Democrat,4
...,...,...,...,...,...,...,...,...
296,reppittenger,NaN,North Carolina,NaN,United States of America,Yes,Republican,428
189,repsandylevin,NaN,Michigan,NaN,United States of America,Yes,Democrat,429
143,rosadelauro,NaN,Connecticut,NaN,United States of America,Yes,Democrat,430
110,sethmoulton,NaN,Massachusetts,NaN,United States of America,Yes,Democrat,431


In [ ]:
usersdf2 = pd.concat([politiciandf, nonpoliticiandf])

In [ ]:
usersdf2

,user_handle,user_followers_count,state,city,country,is_politician,political_affiliation,user_id
146,AGBecerra,NaN,California,NaN,United States of America,Yes,Democrat,0
173,AlanGrayson,NaN,Florida,NaN,United States of America,Yes,Democrat,1
39,AnthonyBrownMD4,NaN,Maryland,NaN,United States of America,Yes,Democrat,2
345,AustinScottGA08,NaN,Georgia,NaN,United States of America,Yes,Republican,3
162,BennieGThompson,NaN,Mississippi,NaN,United States of America,Yes,Democrat,4
...,...,...,...,...,...,...,...,...
781684,zzhuffxx,75.0,California,NaN,United States of America,No,NaN,2545204974.0
781688,zzo_pi,80.0,New York,NaN,United States of America,No,NaN,1.3075129524053238e+18
781689,zzpop_5,7.0,Texas,Austin,United States of America,No,NaN,3158296596.0
781694,zzxyxy,20997.0,California,NaN,United States of America,No,NaN,48883553.0


In [ ]:
politiciandf

,user_handle,user_followers_count,state,city,country,is_politician,political_affiliation,user_id
146,AGBecerra,NaN,California,NaN,United States of America,Yes,Democrat,0
173,AlanGrayson,NaN,Florida,NaN,United States of America,Yes,Democrat,1
39,AnthonyBrownMD4,NaN,Maryland,NaN,United States of America,Yes,Democrat,2
345,AustinScottGA08,NaN,Georgia,NaN,United States of America,Yes,Republican,3
162,BennieGThompson,NaN,Mississippi,NaN,United States of America,Yes,Democrat,4
...,...,...,...,...,...,...,...,...
296,reppittenger,NaN,North Carolina,NaN,United States of America,Yes,Republican,428
189,repsandylevin,NaN,Michigan,NaN,United States of America,Yes,Democrat,429
143,rosadelauro,NaN,Connecticut,NaN,United States of America,Yes,Democrat,430
110,sethmoulton,NaN,Massachusetts,NaN,United States of America,Yes,Democrat,431


In [ ]:
politiciandf.nunique()

user_handle              433
user_followers_count       0
state                     52
city                       0
country                    1
is_politician              1
political_affiliation      2
user_id                  433
dtype: int64

In [ ]:
usersdf2.nunique()

user_handle              129760
user_followers_count      13738
state                       711
city                       1396
country                     129
is_politician                 2
political_affiliation         2
user_id                  129772
dtype: int64

In [ ]:
politiciantweets = pd.read_csv('ExtractedTweets.csv')

In [ ]:
politiciantweets.rename(columns={'Handle' : 'user_handle', 'Tweet' : 'tweet'}, inplace=True)
politiciantweets.head()

,Party,user_handle,tweet
0,Democrat,RepDarrenSoto,"Today, Senate Dems vote to #SaveTheInternet. P..."
1,Democrat,RepDarrenSoto,RT @WinterHavenSun: Winter Haven resident / Al...
2,Democrat,RepDarrenSoto,RT @NBCLatino: .@RepDarrenSoto noted that Hurr...
3,Democrat,RepDarrenSoto,RT @NALCABPolicy: Meeting with @RepDarrenSoto ...
4,Democrat,RepDarrenSoto,RT @Vegalteno: Hurricane season starts on June...


In [ ]:
politiciantweets.drop(columns=['Party'], inplace=True)

In [ ]:
politiciantweets['tweet_id'] = None
politiciantweets['created_at'] = None
politiciantweets['likes'] = None
politiciantweets['retweet_count'] = None
politiciantweets['lat'] = None
politiciantweets['long'] = None
politiciantweets['state'] = None
politiciantweets['state_code'] = None
politiciantweets['country'] = None

In [ ]:
politiciantweets

,user_handle,tweet,tweet_id,created_at,likes,retweet_count,lat,long,state,state_code,country
0,RepDarrenSoto,"Today, Senate Dems vote to #SaveTheInternet. P...",None,None,None,None,None,None,None,None,None
1,RepDarrenSoto,RT @WinterHavenSun: Winter Haven resident / Al...,None,None,None,None,None,None,None,None,None
2,RepDarrenSoto,RT @NBCLatino: .@RepDarrenSoto noted that Hurr...,None,None,None,None,None,None,None,None,None
3,RepDarrenSoto,RT @NALCABPolicy: Meeting with @RepDarrenSoto ...,None,None,None,None,None,None,None,None,None
4,RepDarrenSoto,RT @Vegalteno: Hurricane season starts on June...,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
86455,RepTomPrice,Check out my op-ed on need for End Executive O...,None,None,None,None,None,None,None,None,None
86456,RepTomPrice,"Yesterday, Betty &amp; I had a great time lear...",None,None,None,None,None,None,None,None,None
86457,RepTomPrice,We are forever grateful for the service and sa...,None,None,None,None,None,None,None,None,None
86458,RepTomPrice,Happy first day of school @CobbSchools! #CobbB...,None,None,None,None,None,None,None,None,None


In [ ]:
politiciandf2 = politiciandf.copy()
politiciantweets = pd.merge(politiciantweets, politiciandf2[['user_handle', 'user_id']], on='user_handle', how='left')

In [ ]:
politiciantweets

,user_handle,tweet,tweet_id,created_at,likes,retweet_count,lat,long,state,state_code,country,user_id
0,RepDarrenSoto,"Today, Senate Dems vote to #SaveTheInternet. P...",None,None,None,None,None,None,None,None,None,137
1,RepDarrenSoto,RT @WinterHavenSun: Winter Haven resident / Al...,None,None,None,None,None,None,None,None,None,137
2,RepDarrenSoto,RT @NBCLatino: .@RepDarrenSoto noted that Hurr...,None,None,None,None,None,None,None,None,None,137
3,RepDarrenSoto,RT @NALCABPolicy: Meeting with @RepDarrenSoto ...,None,None,None,None,None,None,None,None,None,137
4,RepDarrenSoto,RT @Vegalteno: Hurricane season starts on June...,None,None,None,None,None,None,None,None,None,137
...,...,...,...,...,...,...,...,...,...,...,...,...
86455,RepTomPrice,Check out my op-ed on need for End Executive O...,None,None,None,None,None,None,None,None,None,342
86456,RepTomPrice,"Yesterday, Betty &amp; I had a great time lear...",None,None,None,None,None,None,None,None,None,342
86457,RepTomPrice,We are forever grateful for the service and sa...,None,None,None,None,None,None,None,None,None,342
86458,RepTomPrice,Happy first day of school @CobbSchools! #CobbB...,None,None,None,None,None,None,None,None,None,342


In [ ]:
final_df

,tweet_id,user_id,user_handle,created_at,tweet,likes,retweet_count,lat,long,state,state_code,country
0,1.316529221557252e+18,360666534.0,elsollatinonews,2020-10-15 00:00:01,#Elecciones2020 | En #Florida: #JoeBiden dice ...,0.0,0.0,25.77427,-80.19366,Florida,FL,United States of America
1,1.316529228091847e+18,8436472.0,snarke,2020-10-15 00:00:02,"#Trump: As a student I used to hear for years,...",2.0,1.0,45.5202471,-122.6741949,Oregon,OR,United States of America
2,1.3165292523014513e+18,47413798.0,Ranaabtar,2020-10-15 00:00:08,You get a tie! And you get a tie! #Trump ‘s ra...,4.0,3.0,38.8949924,-77.0365581,District of Columbia,DC,United States of America
3,1.316529291052675e+18,1138416104.0,FarrisFlagg,2020-10-15 00:00:17,@CLady62 Her 15 minutes were over long time ag...,2.0,0.0,33.7825194,-117.22864779999999,California,CA,United States of America
4,1.3165292934979625e+18,9.007610716314296e+17,sm_gulledge,2020-10-15 00:00:18,@DeeviousDenise @realDonaldTrump @nypost There...,0.0,0.0,40.225356899999994,-82.6881395,Ohio,OH,United States of America
...,...,...,...,...,...,...,...,...,...,...,...,...
580792,1.3255885261178757e+18,4571814261.0,DisguiseDevilin,2020-11-08 23:58:27,#Nationals invite #Biden https://t.co/UVEs3CB6mM,0.0,0.0,8.5322391,38.626471200000005,Oromia Region,NaN,Ethiopia
580793,1.3255886042492723e+18,7.40635438478594e+17,CarlosValenz15,2020-11-08 23:58:46,No puede ser lo #Panazis como siempre sacando ...,0.0,0.0,32.501018800000004,-116.9646629,Baja California,BCN,Mexico
580794,1.3255886120843305e+18,1.2480465028214456e+18,DrJoeMcCarthy,2020-11-08 23:58:48,The man needs some help...#usa #biden\nWhen wi...,0.0,0.0,43.51963,-114.31532,Idaho,ID,United States of America
580795,1.3255886292516127e+18,1.317045546616578e+18,MrErdem23,2020-11-08 23:58:52,Şu #biden geleli 1 gün oldu #maliyebakanı #ist...,0.0,0.0,38.4147331,27.1434119,Izmir,NaN,Turkey


In [ ]:
def can_convert_to_float(value):
    try:
        float(value)
        return True
    except (TypeError, ValueError):
        return False

final_df['can_convert'] = final_df['likes'].apply(can_convert_to_float)

erroneous_rows = final_df[~final_df['can_convert']]
print("Erroneous rows:")
print(erroneous_rows)

final_df = final_df[final_df['can_convert']].copy()

final_df.drop(columns=['can_convert'], inplace=True)

Erroneous rows:
       tweet_id    user_id          user_handle  \
179973      0.0  peraforex  2020-07-14 13:19:26   

                                               created_at tweet  \
179973  Fırsatı Kaçırmayın ! #forex #abdseçim #joebide...   0.0   

                     likes  retweet_count     lat            long  \
179973  Twitter for iPhone   1.283028e+18  London  United Kingdom   

                             state state_code  country  can_convert  
179973  2020-11-04 10:49:34.000962       None  England        False  


In [ ]:
final_df = pd.concat([final_df, politiciantweets])

In [ ]:
final_df.drop(columns=['Party'], inplace=True)

KeyError: "['Party'] not found in axis"

In [ ]:
final_df.nunique()

tweet_id         509929
user_id          145936
user_handle      146124
created_at       405513
tweet            589904
likes              1444
retweet_count       656
lat                6059
long               6061
state               711
state_code          308
country             126
dtype: int64

In [ ]:
final_df['tweet'] = final_df['tweet'].str.replace('&amp;', '&', regex=False)

In [ ]:
final_df

,tweet_id,user_id,user_handle,created_at,tweet,likes,retweet_count,lat,long,state,state_code,country
0,1.316529221557252e+18,360666534.0,elsollatinonews,2020-10-15 00:00:01,#Elecciones2020 | En #Florida: #JoeBiden dice ...,0.0,0.0,25.77427,-80.19366,Florida,FL,United States of America
1,1.316529228091847e+18,8436472.0,snarke,2020-10-15 00:00:02,"#Trump: As a student I used to hear for years,...",2.0,1.0,45.5202471,-122.6741949,Oregon,OR,United States of America
2,1.3165292523014513e+18,47413798.0,Ranaabtar,2020-10-15 00:00:08,You get a tie! And you get a tie! #Trump ‘s ra...,4.0,3.0,38.8949924,-77.0365581,District of Columbia,DC,United States of America
3,1.316529291052675e+18,1138416104.0,FarrisFlagg,2020-10-15 00:00:17,@CLady62 Her 15 minutes were over long time ag...,2.0,0.0,33.7825194,-117.22864779999999,California,CA,United States of America
4,1.3165292934979625e+18,9.007610716314296e+17,sm_gulledge,2020-10-15 00:00:18,@DeeviousDenise @realDonaldTrump @nypost There...,0.0,0.0,40.225356899999994,-82.6881395,Ohio,OH,United States of America
...,...,...,...,...,...,...,...,...,...,...,...,...
86455,None,342,RepTomPrice,None,Check out my op-ed on need for End Executive O...,None,NaN,None,None,None,None,None
86456,None,342,RepTomPrice,None,"Yesterday, Betty & I had a great time learning...",None,NaN,None,None,None,None,None
86457,None,342,RepTomPrice,None,We are forever grateful for the service and sa...,None,NaN,None,None,None,None,None
86458,None,342,RepTomPrice,None,Happy first day of school @CobbSchools! #CobbB...,None,NaN,None,None,None,None,None


In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
final_df['clean_tweet'] = final_df['tweet'].apply(lambda x: ' '.join(word for word in x.split() if not (word.startswith('#') or word.startswith('@') or word.startswith('https'))))

In [ ]:
final_df[:5]

,tweet_id,user_id,user_handle,created_at,tweet,likes,retweet_count,lat,long,state,state_code,country,clean_tweet
0,1.316529221557252e+18,360666534.0,elsollatinonews,2020-10-15 00:00:01,#Elecciones2020 | En #Florida: #JoeBiden dice ...,0.0,0.0,25.77427,-80.19366,Florida,FL,United States of America,| En dice que solo se preocupa por él mismo. E...
1,1.316529228091847e+18,8436472.0,snarke,2020-10-15 00:00:02,"#Trump: As a student I used to hear for years,...",2.0,1.0,45.5202471,-122.6741949,Oregon,OR,United States of America,"As a student I used to hear for years, for ten..."
2,1.3165292523014513e+18,47413798.0,Ranaabtar,2020-10-15 00:00:08,You get a tie! And you get a tie! #Trump ‘s ra...,4.0,3.0,38.8949924,-77.0365581,District of Columbia,DC,United States of America,You get a tie! And you get a tie! ‘s rally
3,1.316529291052675e+18,1138416104.0,FarrisFlagg,2020-10-15 00:00:17,@CLady62 Her 15 minutes were over long time ag...,2.0,0.0,33.7825194,-117.22864779999999,California,CA,United States of America,Her 15 minutes were over long time ago. Omaros...
4,1.3165292934979625e+18,9.007610716314296e+17,sm_gulledge,2020-10-15 00:00:18,@DeeviousDenise @realDonaldTrump @nypost There...,0.0,0.0,40.225356899999994,-82.6881395,Ohio,OH,United States of America,There won’t be many of them. Unless you all ha...


In [ ]:
stop_words = list(set(stopwords.words('english') +
                 stopwords.words('spanish') +
                 stopwords.words('french') +
                 stopwords.words('german')))
vectorizer = CountVectorizer(max_df=0.95, min_df=100, stop_words=stop_words)
tweet_matrix = vectorizer.fit_transform(final_df['clean_tweet'])

vocab = vectorizer.get_feature_names_out()

In [ ]:

example_tweet = final_df['tweet'].iloc[0]
transformed_vector = vectorizer.transform([example_tweet])
tweet_array = transformed_vector.toarray()
words_indices = tweet_array.nonzero()[1]
words = [vocab[i] for i in words_indices]

print("Tokenized words:", words)

Tokenized words: ['aquí', 'co', 'demócrata', 'dice', 'donaldtrump', 'florida', 'https', 'joebiden', 'mismo', 'solo']


In [ ]:
n_topics = 8
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [ ]:
nmf = NMF(n_components=n_topics, random_state=1,  l1_ratio=.5, init='nndsvd')
nmf_topics = nmf.fit_transform(tweet_matrix)

In [ ]:
display_topics(nmf, vocab, 20)
nmf_mapping = {0: 'Trump Campaign', 1: 'Biden Campaign', 2: 'Voting Process', 3: 'Voters / People', 4: 'Office of the President', 5: 'United States of America', 6: 'Speculation / Comparison', 7: 'Election Day'}

Topic 0:
trump donald supporters via 2020 covid anti campaign biden says gop administration rally 2016 please voted white debate said voters
Topic 1:
biden joe hunter harris wins via votes kamala win china obama donald family campaign lead victory news pennsylvania presidente states
Topic 2:
vote please blue count mail voting votes every early get today go anti america popular counting stop make voted ballot
Topic 3:
people get one time would know america rt american today country need years see many want good let go think
Topic 4:
president elect states vice america united congratulations first 46th new elected next joe mr history american harris kamala usa donald
Topic 5:
us rt 2020 world donald presidential elections new let live care help take china democracy today joe americans years see
Topic 6:
like would looks look going know sounds think see never even feel really one years right good said say someone
Topic 7:
election votes win day 2020 presidential states wins going results 

In [ ]:
final_df['NMF_Topic'] = nmf_topics.argmax(axis=1)

In [ ]:
topic_zero_entries = final_df[final_df['NMF_Topic'] == 2]

In [ ]:
final_df['NMF_Topic'].value_counts()

NMF_Topic
3    272041
0     91442
7     64084
1     56515
5     31290
4     30610
6     27495
2     22912
Name: count, dtype: int64

In [ ]:
topic_zero_entries[:20]

,tweet_id,user_id,user_handle,created_at,tweet,likes,retweet_count,lat,long,state,state_code,country,clean_tweet,NMF_Topic
15,1.316529648323289e+18,2818208253.0,RickenRich,2020-10-15 00:01:42,#BlacksForTrump \n#BlackVoicesForTrump \n#Bide...,1.0,1.0,36.7014631,-118.75599740000001,California,CA,United States of America,Vote down ticket & give somebody to work with,2
19,1.3165296917343724e+18,58488519.0,TheLaughMakers,2020-10-15 00:01:53,"#TheWeek: ""#Trump in Penn: ""I saved suburbia. ...",0.0,0.0,34.148398900000004,-118.39618770000001,California,CA,United States of America,"""#Trump in Penn: ""I saved suburbia. I got rid ...",2
29,1.3165299660587008e+18,497299613.0,sbthro,2020-10-15 00:02:58,#tRump is using the DOJ with our tax dollars t...,0.0,0.0,2.8599609,10.1883263,South,SU,Cameroon,is using the DOJ with our tax dollars to retal...,2
101,1.3165320607067914e+18,24269325.0,kimballkan56,2020-10-15 00:11:17,The thought of having to hear the insane ravin...,0.0,0.0,31.8160381,-99.51209859999999,Texas,TX,United States of America,The thought of having to hear the insane ravin...,2
172,1.316534847515525e+18,19580451.0,mlosuna,2020-10-15 00:22:22,#Twitter is blocking certain #CDC data since t...,0.0,0.0,33.197649600000005,-96.6154471,Texas,TX,United States of America,is blocking certain data since they don't agre...,2
202,1.3165358115888538e+18,194650429.0,MoClarker,2020-10-15 00:26:12,@cnn #Trump Stop blaming others for #fouryears...,0.0,0.0,47.00521079999999,-88.96290959999999,Michigan,MI,United States of America,Stop blaming others for of failure. If you can...,2
207,1.3165359792772506e+18,1.2646345201936876e+18,realmacss,2020-10-15 00:26:52,@DrBiden @staceyabrams #Trump said not to take...,0.0,0.0,27.446705600000005,-80.3256056,Florida,FL,United States of America,said not to take chances and vote in person to...,2
228,1.3165367082179502e+18,123281100.0,AndyOstroy,2020-10-15 00:29:45,California 23rd...vote...for... @KimMangone! I...,58.0,26.0,40.7127281,-74.0060152,New York,NY,United States of America,California 23rd...vote...for... It’s time to d...,2
230,1.3165367783088456e+18,4132841.0,mcleod,2020-10-15 00:30:02,Republican #VoterSuppression has no place in o...,0.0,0.0,29.9978185,-83.2221527,Florida,FL,United States of America,Republican has no place in our democracy. VOTE...,2
250,1.3165373128939602e+18,61854283.0,dtcav,2020-10-15 00:32:10,@DearAuntCrabby Thank you! It's a made up stor...,1.0,0.0,39.9527237,-75.16352619999999,Pennsylvania,PA,United States of America,"Thank you! It's a made up story but again, the...",2


In [ ]:
final_df['Topic'] = final_df['NMF_Topic'].map(nmf_mapping)

In [ ]:
final_df.drop(columns=['NMF_Topic'], inplace=True)

In [ ]:
final_df[:50]

,tweet_id,user_id,user_handle,created_at,tweet,likes,retweet_count,lat,long,state,state_code,country,clean_tweet,Topic
0,1.316529221557252e+18,360666534.0,elsollatinonews,2020-10-15 00:00:01,#Elecciones2020 | En #Florida: #JoeBiden dice ...,0.0,0.0,25.77427,-80.19366,Florida,FL,United States of America,| En dice que solo se preocupa por él mismo. E...,United States of America
1,1.316529228091847e+18,8436472.0,snarke,2020-10-15 00:00:02,"#Trump: As a student I used to hear for years,...",2.0,1.0,45.5202471,-122.6741949,Oregon,OR,United States of America,"As a student I used to hear for years, for ten...",Speculation / Comparison
2,1.3165292523014513e+18,47413798.0,Ranaabtar,2020-10-15 00:00:08,You get a tie! And you get a tie! #Trump ‘s ra...,4.0,3.0,38.8949924,-77.0365581,District of Columbia,DC,United States of America,You get a tie! And you get a tie! ‘s rally,Voters / People
3,1.316529291052675e+18,1138416104.0,FarrisFlagg,2020-10-15 00:00:17,@CLady62 Her 15 minutes were over long time ag...,2.0,0.0,33.7825194,-117.22864779999999,California,CA,United States of America,Her 15 minutes were over long time ago. Omaros...,Voters / People
4,1.3165292934979625e+18,9.007610716314296e+17,sm_gulledge,2020-10-15 00:00:18,@DeeviousDenise @realDonaldTrump @nypost There...,0.0,0.0,40.225356899999994,-82.6881395,Ohio,OH,United States of America,There won’t be many of them. Unless you all ha...,Office of the President
5,1.3165293013329183e+18,540476889.0,jamieo33,2020-10-15 00:00:20,One of the single most effective remedies to e...,0.0,0.0,40.9699889,-77.72788309999999,Pennsylvania,PA,United States of America,One of the single most effective remedies to e...,Voters / People
6,1.3165293244182405e+18,19940334.0,changeillinois,2020-10-15 00:00:25,"In 2020, #NYPost is being #censorship #CENSORE...",0.0,0.0,41.875561600000005,-87.6244212,Illinois,IL,United States of America,"In 2020, is being by Twitter to manipulate a U...",Election Day
7,1.3165293286084813e+18,1.2433153463979663e+18,Anchorman_USA,2020-10-15 00:00:26,#Trump #PresidentTrump #Trump2020LandslideVict...,3.0,5.0,32.7174209,-117.16277140000001,California,CA,United States of America,,Trump Campaign
8,1.316529349290586e+18,343956175.0,SebWrites,2020-10-15 00:00:31,@Susan_Hutch @JoeBiden #Ukraine @RealDonaldTru...,2.0,0.0,55.9533456,-3.1883749,Scotland,SCT,United Kingdom,impeachment ‘how’ often & lied!!!,Voters / People
9,1.3165293935781847e+18,343956175.0,SebWrites,2020-10-15 00:00:42,#Ukraine @RealDonaldTrump impeachment #FactChe...,0.0,2.0,55.9533456,-3.1883749,Scotland,SCT,United Kingdom,impeachment ‘how’ often & lied!!!,Voters / People


In [ ]:
#Initialize VADER
sia = SentimentIntensityAnalyzer()

# Function to get sentiment
def get_sentiment(text):
    return sia.polarity_scores(text)['compound']  # Returns the compound score

In [ ]:
from tqdm import tqdm
tqdm.pandas(desc="Analyzing Sentiments")

# Apply sentiment analysis with progress tracking
final_df['sentiment'] = final_df['tweet'].progress_apply(get_sentiment)

# Display the DataFrame with the new sentiment column
final_df[['tweet', 'sentiment']]

Analyzing Sentiments: 100%|██████████| 596389/596389 [05:24<00:00, 1837.58it/s]


,tweet,sentiment
0,#Elecciones2020 | En #Florida: #JoeBiden dice ...,0.0000
1,"#Trump: As a student I used to hear for years,...",0.5905
2,You get a tie! And you get a tie! #Trump ‘s ra...,0.0000
3,@CLady62 Her 15 minutes were over long time ag...,-0.4912
4,@DeeviousDenise @realDonaldTrump @nypost There...,-0.2617
...,...,...
86455,Check out my op-ed on need for End Executive O...,0.0000
86456,"Yesterday, Betty & I had a great time learning...",0.6249
86457,We are forever grateful for the service and sa...,0.4588
86458,Happy first day of school @CobbSchools! #CobbB...,0.6114


In [ ]:
final_df

,tweet_id,user_id,user_handle,created_at,tweet,likes,retweet_count,lat,long,state,state_code,country,clean_tweet,Topic,sentiment
0,1.316529221557252e+18,360666534.0,elsollatinonews,2020-10-15 00:00:01,#Elecciones2020 | En #Florida: #JoeBiden dice ...,0.0,0.0,25.77427,-80.19366,Florida,FL,United States of America,| En dice que solo se preocupa por él mismo. E...,United States of America,0.0000
1,1.316529228091847e+18,8436472.0,snarke,2020-10-15 00:00:02,"#Trump: As a student I used to hear for years,...",2.0,1.0,45.5202471,-122.6741949,Oregon,OR,United States of America,"As a student I used to hear for years, for ten...",Speculation / Comparison,0.5905
2,1.3165292523014513e+18,47413798.0,Ranaabtar,2020-10-15 00:00:08,You get a tie! And you get a tie! #Trump ‘s ra...,4.0,3.0,38.8949924,-77.0365581,District of Columbia,DC,United States of America,You get a tie! And you get a tie! ‘s rally,Voters / People,0.0000
3,1.316529291052675e+18,1138416104.0,FarrisFlagg,2020-10-15 00:00:17,@CLady62 Her 15 minutes were over long time ag...,2.0,0.0,33.7825194,-117.22864779999999,California,CA,United States of America,Her 15 minutes were over long time ago. Omaros...,Voters / People,-0.4912
4,1.3165292934979625e+18,9.007610716314296e+17,sm_gulledge,2020-10-15 00:00:18,@DeeviousDenise @realDonaldTrump @nypost There...,0.0,0.0,40.225356899999994,-82.6881395,Ohio,OH,United States of America,There won’t be many of them. Unless you all ha...,Office of the President,-0.2617
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86455,None,342,RepTomPrice,None,Check out my op-ed on need for End Executive O...,None,NaN,None,None,None,None,None,Check out my op-ed on need for End Executive O...,Voters / People,0.0000
86456,None,342,RepTomPrice,None,"Yesterday, Betty & I had a great time learning...",None,NaN,None,None,None,None,None,"Yesterday, Betty & I had a great time learning...",Voters / People,0.6249
86457,None,342,RepTomPrice,None,We are forever grateful for the service and sa...,None,NaN,None,None,None,None,None,We are forever grateful for the service and sa...,Voters / People,0.4588
86458,None,342,RepTomPrice,None,Happy first day of school @CobbSchools! #CobbB...,None,NaN,None,None,None,None,None,Happy first day of school,Election Day,0.6114


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
final_df = pd.merge(final_df, usersdf2[['user_id', 'political_affiliation']], on='user_id', how='left')

In [ ]:
final_df

,tweet_id,user_id,user_handle,created_at,tweet,likes,retweet_count,lat,long,state,state_code,country,clean_tweet,Topic,sentiment,political_affiliation
0,1.316529221557252e+18,360666534.0,elsollatinonews,2020-10-15 00:00:01,#Elecciones2020 | En #Florida: #JoeBiden dice ...,0.0,0.0,25.77427,-80.19366,Florida,FL,United States of America,| En dice que solo se preocupa por él mismo. E...,United States of America,0.0000,NaN
1,1.316529228091847e+18,8436472.0,snarke,2020-10-15 00:00:02,"#Trump: As a student I used to hear for years,...",2.0,1.0,45.5202471,-122.6741949,Oregon,OR,United States of America,"As a student I used to hear for years, for ten...",Speculation / Comparison,0.5905,NaN
2,1.3165292523014513e+18,47413798.0,Ranaabtar,2020-10-15 00:00:08,You get a tie! And you get a tie! #Trump ‘s ra...,4.0,3.0,38.8949924,-77.0365581,District of Columbia,DC,United States of America,You get a tie! And you get a tie! ‘s rally,Voters / People,0.0000,NaN
3,1.316529291052675e+18,1138416104.0,FarrisFlagg,2020-10-15 00:00:17,@CLady62 Her 15 minutes were over long time ag...,2.0,0.0,33.7825194,-117.22864779999999,California,CA,United States of America,Her 15 minutes were over long time ago. Omaros...,Voters / People,-0.4912,NaN
4,1.3165292934979625e+18,9.007610716314296e+17,sm_gulledge,2020-10-15 00:00:18,@DeeviousDenise @realDonaldTrump @nypost There...,0.0,0.0,40.225356899999994,-82.6881395,Ohio,OH,United States of America,There won’t be many of them. Unless you all ha...,Office of the President,-0.2617,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
596384,None,342,RepTomPrice,None,Check out my op-ed on need for End Executive O...,None,NaN,None,None,None,None,None,Check out my op-ed on need for End Executive O...,Voters / People,0.0000,Republican
596385,None,342,RepTomPrice,None,"Yesterday, Betty & I had a great time learning...",None,NaN,None,None,None,None,None,"Yesterday, Betty & I had a great time learning...",Voters / People,0.6249,Republican
596386,None,342,RepTomPrice,None,We are forever grateful for the service and sa...,None,NaN,None,None,None,None,None,We are forever grateful for the service and sa...,Voters / People,0.4588,Republican
596387,None,342,RepTomPrice,None,Happy first day of school @CobbSchools! #CobbB...,None,NaN,None,None,None,None,None,Happy first day of school,Election Day,0.6114,Republican


In [ ]:
politician_tweets = final_df.dropna(subset=['political_affiliation'])

In [ ]:
politician_tweets['label'] = politician_tweets['political_affiliation'].map({'Democrat': 1, 'Republican': 0})

<ipython-input-789-8b7ebb24ff18>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  politician_tweets['label'] = politician_tweets['political_affiliation'].map({'Democrat': 1, 'Republican': 0})


In [ ]:
politician_tweets

,tweet_id,user_id,user_handle,created_at,tweet,likes,retweet_count,lat,long,state,state_code,country,clean_tweet,Topic,sentiment,political_affiliation,label
509929,None,137,RepDarrenSoto,None,"Today, Senate Dems vote to #SaveTheInternet. P...",None,NaN,None,None,None,None,None,"Today, Senate Dems vote to Proud to support si...",Voting Process,0.7003,Democrat,1
509930,None,137,RepDarrenSoto,None,RT @WinterHavenSun: Winter Haven resident / Al...,None,NaN,None,None,None,None,None,RT Winter Haven resident / Alta Vista teacher ...,Voters / People,0.0000,Democrat,1
509931,None,137,RepDarrenSoto,None,RT @NBCLatino: .@RepDarrenSoto noted that Hurr...,None,NaN,None,None,None,None,None,RT .@RepDarrenSoto noted that Hurricane Maria ...,Voters / People,-0.4404,Democrat,1
509932,None,137,RepDarrenSoto,None,RT @NALCABPolicy: Meeting with @RepDarrenSoto ...,None,NaN,None,None,None,None,None,RT Meeting with . Thanks for taking the time t...,Voters / People,0.4404,Democrat,1
509933,None,137,RepDarrenSoto,None,RT @Vegalteno: Hurricane season starts on June...,None,NaN,None,None,None,None,None,RT Hurricane season starts on June 1st; Puerto...,Voters / People,0.0000,Democrat,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
596384,None,342,RepTomPrice,None,Check out my op-ed on need for End Executive O...,None,NaN,None,None,None,None,None,Check out my op-ed on need for End Executive O...,Voters / People,0.0000,Republican,0
596385,None,342,RepTomPrice,None,"Yesterday, Betty & I had a great time learning...",None,NaN,None,None,None,None,None,"Yesterday, Betty & I had a great time learning...",Voters / People,0.6249,Republican,0
596386,None,342,RepTomPrice,None,We are forever grateful for the service and sa...,None,NaN,None,None,None,None,None,We are forever grateful for the service and sa...,Voters / People,0.4588,Republican,0
596387,None,342,RepTomPrice,None,Happy first day of school @CobbSchools! #CobbB...,None,NaN,None,None,None,None,None,Happy first day of school,Election Day,0.6114,Republican,0


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(politician_tweets['tweet'], politician_tweets['label'], test_size=0.1)

In [ ]:
pipeline = make_pipeline(
    TfidfVectorizer(stop_words=stop_words),
    LogisticRegression()
)

pipeline.fit(X_train, y_train)
pipeline.predict(X_test)

y_pred = pipeline.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

report = classification_report(y_test, y_pred)
print("Classification Report:\n", report)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.78
Classification Report:
               precision    recall  f1-score   support

           0       0.77      0.82      0.79      4457
           1       0.79      0.74      0.77      4189

    accuracy                           0.78      8646
   macro avg       0.78      0.78      0.78      8646
weighted avg       0.78      0.78      0.78      8646



In [ ]:
pipeline = make_pipeline(
    TfidfVectorizer(stop_words=stop_words),
    LogisticRegression()
)
pipeline.fit(politician_tweets['tweet'], politician_tweets['label'])

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('tfidfvectorizer',
                 TfidfVectorizer(stop_words=['mi', 'tenía', 'ihrem', 'dann',
                                             'wieder', 'seamos', 'jenem',
                                             'fueras', 'je', 'whom', 'quien',
                                             'fuerais', 'wir', 'votre', 'pero',
                                             'up', 'has', 'am', 'diesen',
                                             'tuviera', 'solcher',
                                             'estuvieseis', 'es', 'à', 'jetzt',
                                             've', 'fut', 'those', 'manche',
                                             'during', ...])),
                ('logisticregression', LogisticRegression())])

In [ ]:
probabilities = pipeline.predict_proba(final_df['tweet'])
final_df['bias'] = probabilities[:, 1] - probabilities[:, 0]


In [ ]:
politician_tweets.value_counts('label')

label
0    44392
1    42068
Name: count, dtype: int64

In [ ]:
final_df.value_counts('bias')

bias
0.983124    8271
0.572146    5258
0.747886    4621
0.994780    2467
0.101941    1607
            ... 
0.444355       1
0.444355       1
0.444342       1
0.444341       1
0.601689       1
Name: count, Length: 511880, dtype: int64

In [ ]:
final_df[:50]

,tweet_id,user_id,user_handle,created_at,tweet,likes,retweet_count,lat,long,state,state_code,country,clean_tweet,Topic,sentiment,political_affiliation,bias
0,1.316529221557252e+18,360666534.0,elsollatinonews,2020-10-15 00:00:01,#Elecciones2020 | En #Florida: #JoeBiden dice ...,0.0,0.0,25.77427,-80.19366,Florida,FL,United States of America,| En dice que solo se preocupa por él mismo. E...,United States of America,0.0000,NaN,0.352071
1,1.316529228091847e+18,8436472.0,snarke,2020-10-15 00:00:02,"#Trump: As a student I used to hear for years,...",2.0,1.0,45.5202471,-122.6741949,Oregon,OR,United States of America,"As a student I used to hear for years, for ten...",Speculation / Comparison,0.5905,NaN,0.629671
2,1.3165292523014513e+18,47413798.0,Ranaabtar,2020-10-15 00:00:08,You get a tie! And you get a tie! #Trump ‘s ra...,4.0,3.0,38.8949924,-77.0365581,District of Columbia,DC,United States of America,You get a tie! And you get a tie! ‘s rally,Voters / People,0.0000,NaN,0.555917
3,1.316529291052675e+18,1138416104.0,FarrisFlagg,2020-10-15 00:00:17,@CLady62 Her 15 minutes were over long time ag...,2.0,0.0,33.7825194,-117.22864779999999,California,CA,United States of America,Her 15 minutes were over long time ago. Omaros...,Voters / People,-0.4912,NaN,0.896543
4,1.3165292934979625e+18,9.007610716314296e+17,sm_gulledge,2020-10-15 00:00:18,@DeeviousDenise @realDonaldTrump @nypost There...,0.0,0.0,40.225356899999994,-82.6881395,Ohio,OH,United States of America,There won’t be many of them. Unless you all ha...,Office of the President,-0.2617,NaN,0.371221
5,1.3165293013329183e+18,540476889.0,jamieo33,2020-10-15 00:00:20,One of the single most effective remedies to e...,0.0,0.0,40.9699889,-77.72788309999999,Pennsylvania,PA,United States of America,One of the single most effective remedies to e...,Voters / People,0.5256,NaN,0.144638
6,1.3165293244182405e+18,19940334.0,changeillinois,2020-10-15 00:00:25,"In 2020, #NYPost is being #censorship #CENSORE...",0.0,0.0,41.875561600000005,-87.6244212,Illinois,IL,United States of America,"In 2020, is being by Twitter to manipulate a U...",Election Day,-0.4137,NaN,0.707549
7,1.3165293286084813e+18,1.2433153463979663e+18,Anchorman_USA,2020-10-15 00:00:26,#Trump #PresidentTrump #Trump2020LandslideVict...,3.0,5.0,32.7174209,-117.16277140000001,California,CA,United States of America,,Trump Campaign,0.0000,NaN,0.168693
8,1.316529349290586e+18,343956175.0,SebWrites,2020-10-15 00:00:31,@Susan_Hutch @JoeBiden #Ukraine @RealDonaldTru...,2.0,0.0,55.9533456,-3.1883749,Scotland,SCT,United Kingdom,impeachment ‘how’ often & lied!!!,Voters / People,-0.5386,NaN,0.961063
9,1.3165293935781847e+18,343956175.0,SebWrites,2020-10-15 00:00:42,#Ukraine @RealDonaldTrump impeachment #FactChe...,0.0,2.0,55.9533456,-3.1883749,Scotland,SCT,United Kingdom,impeachment ‘how’ often & lied!!!,Voters / People,-0.5386,NaN,0.922337


In [ ]:
final_df['bias'].mean()

0.5087924535182659

In [ ]:
final_df['bias'] = final_df['bias'].round(4)


In [ ]:
final_df

,tweet_id,user_id,user_handle,created_at,tweet,likes,retweet_count,lat,long,state,state_code,country,clean_tweet,Topic,sentiment,political_affiliation,bias
0,1.316529221557252e+18,360666534.0,elsollatinonews,2020-10-15 00:00:01,#Elecciones2020 | En #Florida: #JoeBiden dice ...,0.0,0.0,25.77427,-80.19366,Florida,FL,United States of America,| En dice que solo se preocupa por él mismo. E...,United States of America,0.0000,NaN,0.3521
1,1.316529228091847e+18,8436472.0,snarke,2020-10-15 00:00:02,"#Trump: As a student I used to hear for years,...",2.0,1.0,45.5202471,-122.6741949,Oregon,OR,United States of America,"As a student I used to hear for years, for ten...",Speculation / Comparison,0.5905,NaN,0.6297
2,1.3165292523014513e+18,47413798.0,Ranaabtar,2020-10-15 00:00:08,You get a tie! And you get a tie! #Trump ‘s ra...,4.0,3.0,38.8949924,-77.0365581,District of Columbia,DC,United States of America,You get a tie! And you get a tie! ‘s rally,Voters / People,0.0000,NaN,0.5559
3,1.316529291052675e+18,1138416104.0,FarrisFlagg,2020-10-15 00:00:17,@CLady62 Her 15 minutes were over long time ag...,2.0,0.0,33.7825194,-117.22864779999999,California,CA,United States of America,Her 15 minutes were over long time ago. Omaros...,Voters / People,-0.4912,NaN,0.8965
4,1.3165292934979625e+18,9.007610716314296e+17,sm_gulledge,2020-10-15 00:00:18,@DeeviousDenise @realDonaldTrump @nypost There...,0.0,0.0,40.225356899999994,-82.6881395,Ohio,OH,United States of America,There won’t be many of them. Unless you all ha...,Office of the President,-0.2617,NaN,0.3712
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
596384,None,342,RepTomPrice,None,Check out my op-ed on need for End Executive O...,None,NaN,None,None,None,None,None,Check out my op-ed on need for End Executive O...,Voters / People,0.0000,Republican,-0.8362
596385,None,342,RepTomPrice,None,"Yesterday, Betty & I had a great time learning...",None,NaN,None,None,None,None,None,"Yesterday, Betty & I had a great time learning...",Voters / People,0.6249,Republican,-0.3705
596386,None,342,RepTomPrice,None,We are forever grateful for the service and sa...,None,NaN,None,None,None,None,None,We are forever grateful for the service and sa...,Voters / People,0.4588,Republican,-0.5152
596387,None,342,RepTomPrice,None,Happy first day of school @CobbSchools! #CobbB...,None,NaN,None,None,None,None,None,Happy first day of school,Election Day,0.6114,Republican,-0.3167


In [ ]:
state_to_code = final_df.set_index('state')['state_code'].to_dict()

In [ ]:
state_to_code

{'Florida': 'FL',
 'Oregon': 'OR',
 'District of Columbia': 'DC',
 'California': 'CA',
 'Ohio': 'OH',
 'Pennsylvania': 'PA',
 'Illinois': 'IL',
 'Scotland': 'SCT',
 'Michigan': 'MI',
 'New Jersey': 'NJ',
 'England': 'ENG',
 'Massachusetts': 'MA',
 'New Hampshire': 'NH',
 'South': 'SU',
 'Texas': 'TX',
 'Limburg': 'LI',
 'South Dakota': 'SD',
 'Georgia': 'GA',
 'Missouri': 'MO',
 'New York': 'NY',
 'Colorado': 'CO',
 'South Carolina': 'SC',
 'Sonora': 'SON',
 'Ontario': 'ON',
 'Virginia': 'VA',
 'Alabama': 'AL',
 'Ile-de-France': 'IDF',
 'New South Wales': 'NSW',
 'Arizona': 'AZ',
 'North Carolina': 'NC',
 'Jeollanam-do': nan,
 'Western Australia': 'WA',
 'Tennessee': 'TN',
 'Nebraska': 'NE',
 'Uusimaa': nan,
 'Queensland': 'QLD',
 'Delhi': 'DL',
 'Baden-Württemberg': 'BW',
 'Louisiana': 'LA',
 'Zurich': 'ZH',
 'Nevada': 'NV',
 'Minnesota': 'MN',
 'Indiana': 'IN',
 'Rhineland-Palatinate': 'RP',
 'Tamaulipas': 'TAM',
 'Washington': 'WA',
 'Alberta': 'AB',
 'Hawaii': 'HI',
 'West Virginia

In [ ]:
politician_states = usersdf2[usersdf2['is_politician'] == 'Yes'].set_index('user_id')['state'].to_dict()

In [ ]:
state_to_code['Virgin Islands'] = 'VI'

In [ ]:
for idx, row in final_df.iterrows():
    user_id = row['user_id']
    if user_id in politician_states and politician_states[user_id] in state_to_code:
        final_df.at[idx, 'state'] = politician_states[user_id]
        final_df.at[idx, 'state_code'] = state_to_code[politician_states[user_id]]
        final_df.at[idx, 'country'] = 'United States of America'


In [ ]:
final_df

,tweet_id,user_id,user_handle,created_at,tweet,likes,retweet_count,lat,long,state,state_code,country,clean_tweet,Topic,sentiment,political_affiliation,bias
0,1.316529221557252e+18,360666534.0,elsollatinonews,2020-10-15 00:00:01,#Elecciones2020 | En #Florida: #JoeBiden dice ...,0.0,0.0,25.77427,-80.19366,Florida,FL,United States of America,| En dice que solo se preocupa por él mismo. E...,United States of America,0.0000,NaN,0.3521
1,1.316529228091847e+18,8436472.0,snarke,2020-10-15 00:00:02,"#Trump: As a student I used to hear for years,...",2.0,1.0,45.5202471,-122.6741949,Oregon,OR,United States of America,"As a student I used to hear for years, for ten...",Speculation / Comparison,0.5905,NaN,0.6297
2,1.3165292523014513e+18,47413798.0,Ranaabtar,2020-10-15 00:00:08,You get a tie! And you get a tie! #Trump ‘s ra...,4.0,3.0,38.8949924,-77.0365581,District of Columbia,DC,United States of America,You get a tie! And you get a tie! ‘s rally,Voters / People,0.0000,NaN,0.5559
3,1.316529291052675e+18,1138416104.0,FarrisFlagg,2020-10-15 00:00:17,@CLady62 Her 15 minutes were over long time ag...,2.0,0.0,33.7825194,-117.22864779999999,California,CA,United States of America,Her 15 minutes were over long time ago. Omaros...,Voters / People,-0.4912,NaN,0.8965
4,1.3165292934979625e+18,9.007610716314296e+17,sm_gulledge,2020-10-15 00:00:18,@DeeviousDenise @realDonaldTrump @nypost There...,0.0,0.0,40.225356899999994,-82.6881395,Ohio,OH,United States of America,There won’t be many of them. Unless you all ha...,Office of the President,-0.2617,NaN,0.3712
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
596384,None,342,RepTomPrice,None,Check out my op-ed on need for End Executive O...,None,NaN,None,None,Georgia,GA,United States of America,Check out my op-ed on need for End Executive O...,Voters / People,0.0000,Republican,-0.8362
596385,None,342,RepTomPrice,None,"Yesterday, Betty & I had a great time learning...",None,NaN,None,None,Georgia,GA,United States of America,"Yesterday, Betty & I had a great time learning...",Voters / People,0.6249,Republican,-0.3705
596386,None,342,RepTomPrice,None,We are forever grateful for the service and sa...,None,NaN,None,None,Georgia,GA,United States of America,We are forever grateful for the service and sa...,Voters / People,0.4588,Republican,-0.5152
596387,None,342,RepTomPrice,None,Happy first day of school @CobbSchools! #CobbB...,None,NaN,None,None,Georgia,GA,United States of America,Happy first day of school,Election Day,0.6114,Republican,-0.3167


In [ ]:
final_df.drop(columns=['clean_tweet', 'political_affiliation'], inplace=True)

In [ ]:
final_df.rename(columns={'Topic': 'topic'}, inplace=True)

In [ ]:
final_df.isnull().sum()

tweet_id         86460
user_id              0
user_handle          0
created_at       86460
tweet                0
likes            86460
retweet_count    86460
lat              86460
long             86460
state             3999
state_code       33609
country           3999
topic                0
sentiment            0
bias                 0
dtype: int64

In [ ]:
mask = final_df['user_id'].isin(usersdf2['user_id'])
final_df = final_df[mask]

In [ ]:
final_df['lat'] = pd.to_numeric(final_df['lat'], errors='coerce')
final_df['long'] = pd.to_numeric(final_df['long'], errors='coerce')
final_df['likes'] = pd.to_numeric(final_df['likes'], errors='coerce')
final_df['retweet_count'] = pd.to_numeric(final_df['retweet_count'], errors='coerce')
final_df['lat'] = final_df['lat'].round(4)
final_df['long'] = final_df['long'].round(4)


<ipython-input-811-012665708f02>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['lat'] = pd.to_numeric(final_df['lat'], errors='coerce')
<ipython-input-811-012665708f02>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['long'] = pd.to_numeric(final_df['long'], errors='coerce')
<ipython-input-811-012665708f02>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

In [ ]:
final_df

,tweet_id,user_id,user_handle,created_at,tweet,likes,retweet_count,lat,long,state,state_code,country,topic,sentiment,bias
0,1.316529221557252e+18,360666534.0,elsollatinonews,2020-10-15 00:00:01,#Elecciones2020 | En #Florida: #JoeBiden dice ...,0.0,0.0,25.7743,-80.1937,Florida,FL,United States of America,United States of America,0.0000,0.3521
1,1.316529228091847e+18,8436472.0,snarke,2020-10-15 00:00:02,"#Trump: As a student I used to hear for years,...",2.0,1.0,45.5202,-122.6742,Oregon,OR,United States of America,Speculation / Comparison,0.5905,0.6297
2,1.3165292523014513e+18,47413798.0,Ranaabtar,2020-10-15 00:00:08,You get a tie! And you get a tie! #Trump ‘s ra...,4.0,3.0,38.8950,-77.0366,District of Columbia,DC,United States of America,Voters / People,0.0000,0.5559
3,1.316529291052675e+18,1138416104.0,FarrisFlagg,2020-10-15 00:00:17,@CLady62 Her 15 minutes were over long time ag...,2.0,0.0,33.7825,-117.2286,California,CA,United States of America,Voters / People,-0.4912,0.8965
4,1.3165292934979625e+18,9.007610716314296e+17,sm_gulledge,2020-10-15 00:00:18,@DeeviousDenise @realDonaldTrump @nypost There...,0.0,0.0,40.2254,-82.6881,Ohio,OH,United States of America,Office of the President,-0.2617,0.3712
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
596384,None,342,RepTomPrice,None,Check out my op-ed on need for End Executive O...,NaN,NaN,NaN,NaN,Georgia,GA,United States of America,Voters / People,0.0000,-0.8362
596385,None,342,RepTomPrice,None,"Yesterday, Betty & I had a great time learning...",NaN,NaN,NaN,NaN,Georgia,GA,United States of America,Voters / People,0.6249,-0.3705
596386,None,342,RepTomPrice,None,We are forever grateful for the service and sa...,NaN,NaN,NaN,NaN,Georgia,GA,United States of America,Voters / People,0.4588,-0.5152
596387,None,342,RepTomPrice,None,Happy first day of school @CobbSchools! #CobbB...,NaN,NaN,NaN,NaN,Georgia,GA,United States of America,Election Day,0.6114,-0.3167


In [ ]:
usersdf2

,user_handle,user_followers_count,is_politician,political_affiliation,user_id
146,AGBecerra,NaN,Yes,Democrat,0
173,AlanGrayson,NaN,Yes,Democrat,1
39,AnthonyBrownMD4,NaN,Yes,Democrat,2
345,AustinScottGA08,NaN,Yes,Republican,3
162,BennieGThompson,NaN,Yes,Democrat,4
...,...,...,...,...,...
781684,zzhuffxx,75.0,No,NaN,2545204974.0
781688,zzo_pi,80.0,No,NaN,1.3075129524053238e+18
781689,zzpop_5,7.0,No,NaN,3158296596.0
781694,zzxyxy,20997.0,No,NaN,48883553.0


In [ ]:
final_df.nunique()

tweet_id         490139
user_id          129772
user_handle      129959
created_at       395102
tweet            570195
likes              1437
retweet_count       653
lat                5940
long               5975
state               711
state_code          309
country             126
topic                 8
sentiment         14504
bias              19864
dtype: int64

In [ ]:
usersdf2.nunique()

user_handle              129760
user_followers_count      13738
state                       711
city                       1396
country                     129
is_politician                 2
political_affiliation         2
user_id                  129772
dtype: int64

In [ ]:
usersdf2.drop(columns=['state', 'city', 'country'], inplace=True)

In [ ]:
final_df.isnull().sum()

tweet_id         86460
user_id              0
user_handle          0
created_at       86460
tweet                0
likes            86460
retweet_count    86460
lat              86460
long             86460
state             3999
state_code       32044
country           3999
topic                0
sentiment            0
bias                 0
dtype: int64

In [ ]:
final_df[final_df['country'].isna()].head(500)

,tweet_id,user_id,user_handle,created_at,tweet,likes,retweet_count,lat,long,state,state_code,country,topic,sentiment,bias
554597,None,404,housebudgetGOP,None,Reconciliation is a powerful #budget tool that...,NaN,NaN,NaN,NaN,None,None,None,Speculation / Comparison,0.7269,-0.7718
554598,None,404,housebudgetGOP,None,"This Week's #BudgetFact: According to @USCBO, ...",NaN,NaN,NaN,NaN,None,None,None,Election Day,0.0000,-0.2014
554599,None,404,housebudgetGOP,None,"WATCH: In this week's #MemberSpotlight, Budget...",NaN,NaN,NaN,NaN,None,None,None,Voters / People,0.0000,-0.7946
554600,None,404,housebudgetGOP,None,"ICYMI: This morning, the committee held its an...",NaN,NaN,NaN,NaN,None,None,None,Election Day,0.0000,-0.9630
554601,None,404,housebudgetGOP,None,HAPPENING NOW! the committee will host its ann...,NaN,NaN,NaN,NaN,None,None,None,Election Day,0.0000,-0.5801
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
563892,None,43,HouseJudiciary,None,Chairman @RepGoodlatte praises House Leadershi...,NaN,NaN,NaN,NaN,None,None,None,Voting Process,0.5267,-0.8597
563893,None,43,HouseJudiciary,None,"Chairman @RepGoodlatte: “Today, President Tru...",NaN,NaN,NaN,NaN,None,None,None,Office of the President,0.3818,-0.6501
563894,None,43,HouseJudiciary,None,Chairman @RepGoodlatte applauds @POTUS for sig...,NaN,NaN,NaN,NaN,None,None,None,Voters / People,0.2500,-0.9385
563895,None,43,HouseJudiciary,None,“While we continue to believe the appointment ...,NaN,NaN,NaN,NaN,None,None,None,Voters / People,0.4019,-0.1921


In [ ]:
null_indices = final_df[final_df['tweet_id'].isna()].index
replacement_values = np.arange(len(null_indices))
final_df.loc[null_indices, 'tweet_id'] = replacement_values

In [ ]:
final_df.nunique()

tweet_id         576599
user_id          129772
user_handle      129959
created_at       395102
tweet            570195
likes              1437
retweet_count       653
lat                5940
long               5975
state               711
state_code          309
country             126
topic                 8
sentiment         14504
bias              19864
dtype: int64

In [ ]:
final_df.isnull().sum()

tweet_id             0
user_id              0
user_handle          0
created_at       86460
tweet                0
likes            86460
retweet_count    86460
lat              86460
long             86460
state             3999
state_code       32044
country           3999
topic                0
sentiment            0
bias                 0
dtype: int64

In [ ]:
usersdf2.nunique()

user_handle              129760
user_followers_count      13738
is_politician                 2
political_affiliation         2
user_id                  129772
dtype: int64

In [ ]:
usersdf2.isnull().sum()

user_handle                   0
user_followers_count        433
is_politician                 0
political_affiliation    129339
user_id                       0
dtype: int64

In [ ]:
final_df

,tweet_id,user_id,user_handle,created_at,tweet,likes,retweet_count,lat,long,state,state_code,country,topic,sentiment,bias
0,1.316529221557252e+18,360666534.0,elsollatinonews,2020-10-15 00:00:01,#Elecciones2020 | En #Florida: #JoeBiden dice ...,0.0,0.0,25.7743,-80.1937,Florida,FL,United States of America,United States of America,0.0000,0.3521
1,1.316529228091847e+18,8436472.0,snarke,2020-10-15 00:00:02,"#Trump: As a student I used to hear for years,...",2.0,1.0,45.5202,-122.6742,Oregon,OR,United States of America,Speculation / Comparison,0.5905,0.6297
2,1.3165292523014513e+18,47413798.0,Ranaabtar,2020-10-15 00:00:08,You get a tie! And you get a tie! #Trump ‘s ra...,4.0,3.0,38.8950,-77.0366,District of Columbia,DC,United States of America,Voters / People,0.0000,0.5559
3,1.316529291052675e+18,1138416104.0,FarrisFlagg,2020-10-15 00:00:17,@CLady62 Her 15 minutes were over long time ag...,2.0,0.0,33.7825,-117.2286,California,CA,United States of America,Voters / People,-0.4912,0.8965
4,1.3165292934979625e+18,9.007610716314296e+17,sm_gulledge,2020-10-15 00:00:18,@DeeviousDenise @realDonaldTrump @nypost There...,0.0,0.0,40.2254,-82.6881,Ohio,OH,United States of America,Office of the President,-0.2617,0.3712
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
596384,86455,342,RepTomPrice,None,Check out my op-ed on need for End Executive O...,NaN,NaN,NaN,NaN,Georgia,GA,United States of America,Voters / People,0.0000,-0.8362
596385,86456,342,RepTomPrice,None,"Yesterday, Betty & I had a great time learning...",NaN,NaN,NaN,NaN,Georgia,GA,United States of America,Voters / People,0.6249,-0.3705
596386,86457,342,RepTomPrice,None,We are forever grateful for the service and sa...,NaN,NaN,NaN,NaN,Georgia,GA,United States of America,Voters / People,0.4588,-0.5152
596387,86458,342,RepTomPrice,None,Happy first day of school @CobbSchools! #CobbB...,NaN,NaN,NaN,NaN,Georgia,GA,United States of America,Election Day,0.6114,-0.3167


In [ ]:
usersdf2.to_csv('usersdf2.csv', index=False)
final_df.to_csv('final_df.csv', index=False)

In [ ]:
from google.colab import files
files.download('final_df.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>